<a href="https://colab.research.google.com/github/Shahahaha11/LION_DEN/blob/main/trial2_DATA_CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name map


---


trial -> trying new variable

---


data_cleaning -> a separate file which ends with saving a pickle that you use in model.ipynb


---


model -> Main model with little extra code

---


model_testing -> Less code, optimized for testing

---


In [1]:
import pandas as pd
import numpy as np
file_path= "/content/austinHousingData.csv"
data = pd.read_csv(file_path)

In [2]:
data.columns

Index(['zpid', 'city', 'streetAddress', 'zipcode', 'description', 'latitude',
       'longitude', 'propertyTaxRate', 'garageSpaces', 'hasAssociation',
       'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa', 'hasView',
       'homeType', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'latestPriceSource', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'lotSizeSqFt', 'livingAreaSqFt',
       'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools',
       'numOfHighSchools', 'avgSchoolDistance', 'avgSchoolRating',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBathrooms',
       'numOfBedrooms', 'numOfStories', 'homeImage'],
      dtype='object')

In [3]:
################ NEW VARIABLE TEST ######### TO MERGE ON ZIP ##############################

In [4]:
house_inventory = pd.read_csv("/content/Affordable_Housing_Inventory_20250327.csv")
house_inventory.columns

Index(['Contract_ID', 'Project_ID', 'Project_Name', 'Ownership', 'Developer',
       'Address', 'ZIP', 'Parcel_ID', 'Council_District', 'Unit_Type',
       'Housing_Tenure', 'Development_Status', 'Total_Units',
       'Affordable_Units', 'Total_ADA_Units', 'PSH_Units', 'MFI_20', 'MFI_30',
       'MFI_40', 'MFI_50', 'MFI_60', 'MFI_65', 'MFI_70', 'MFI_80', 'MFI_100',
       'MFI_120', 'Affordability_Start_Date', 'Affordability_Expiration_Date',
       'Affordability_Period', 'FIL_Status', 'CAL_FIL', 'Date_FIL_Recorded',
       'HDA', 'Private_Activity_Bonds', 'Affordability_Unlocked', 'SMART',
       'Dev_Agreement', 'Density_Bonus', 'FIL', 'LIHTC', 'GO_Bonds_2006',
       'GO_Bonds_2013', 'GO_Bonds_2018', 'GO_Bonds_2022', 'CDBG', 'CIP_Funds',
       'HOME', 'HTF', 'Project_Connect', 'City_Subsidy', 'Longitude',
       'Latitude'],
      dtype='object')

In [5]:
########### CHECKING IF MERGING IS RIGHT #########
missing_count =(data['zipcode'].isin(house_inventory['ZIP']) == False).sum()
print(missing_count)


2032


if we have n unique values in one of the columns in the new data set


In [6]:
house_inventory['ZIP_int'] = house_inventory['ZIP'].dropna().astype(int)
house_inventory['ZIP_int']

,ZIP_int
0,78722.0
1,78745.0
2,78704.0
3,78704.0
4,78704.0
...,...
2370,78744.0
2371,7857.0
2372,78723.0
2373,78653.0


In [7]:
############### DUPLICATED LONGITUDE AND LATITUDE IN HOUSE_INVENTORY ################## FATAL ##########
lat_long_counts = house_inventory.groupby(['Latitude', 'Longitude']).size().reset_index(name='count')
duplicates = lat_long_counts[lat_long_counts['count'] > 1]
print(duplicates)

       Latitude  Longitude  count
0     30.089704 -97.791678      2
4     30.118147 -97.810538     24
7     30.124392 -97.807296      4
119   30.140566 -97.765717      2
207   30.153442 -97.753764      6
...         ...        ...    ...
1919  30.336559 -97.707100      6
1920  30.336560 -97.707100     13
1927  30.340805 -97.739455      3
1960  30.364212 -97.726877      2
2010  30.400555 -97.717067      2

[118 rows x 3 columns]


In [10]:
house_inventory_unique = house_inventory.drop_duplicates(subset='ZIP', keep='first')
print(house_inventory_unique.shape)

(45, 53)


In [11]:
data= data[data['city'] == 'austin']

In [14]:
################# MOCK MERGE , BECAUSE DUPLICATED ZIPS HAVE DIFFERENT CROSS SECTIONAL DATA, WE ARE ONLY TAKING THE FIRST DUPLICATE
############### IMPLICATION : THE DATA IS A REPRESENTATION OF WHAT TYPE OF PROPERTY IS AT THAT ZIPCODE,
######### WE ARE NOT TAKING INTO ACCOUNT OTHER PROPERTIES IN THAT ZIPCODE BUT ONLY THE FIRST

data['Housing_Tenure'] = data['zipcode'].map(
    house_inventory_unique.set_index('ZIP')['Housing_Tenure']
)
print(data.shape)
print(data.isna().sum())

(15020, 48)
zpid                             0
city                             0
streetAddress                    0
zipcode                          0
description                      2
latitude                         0
longitude                        0
propertyTaxRate                  0
garageSpaces                     0
hasAssociation                   0
hasCooling                       0
hasGarage                        0
hasHeating                       0
hasSpa                           0
hasView                          0
homeType                         0
parkingSpaces                    0
yearBuilt                        0
latestPrice                      0
numPriceChanges                  0
latest_saledate                  0
latest_salemonth                 0
latest_saleyear                  0
latestPriceSource                0
numOfPhotos                      0
numOfAccessibilityFeatures       0
numOfAppliances                  0
numOfParkingFeatures             0
numOfPat

In [15]:
#USELESS FEATURES ## WE DROP USED FEATURES ONLY, "city" (used) and "zipcode" (unused) .
useless_features= ['zpid', 'city', 'streetAddress', 'description','homeImage','homeType', 'latestPriceSource']

In [16]:
################# FINAL DATAFRAME IN USE ################
data = data.drop(columns=useless_features, errors='ignore')
data.isna().sum()

,0
zipcode,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0


In [17]:
####### WE DROP AFTER REMOVING USELESS ###
data = data.dropna()
data.shape

(13005, 41)

In [18]:
############ FEATURE ENGINEERING ################
import pandas as pd
def create_dummies(df, column):
  if df[column].nunique() <= 7 :
    return pd.get_dummies(df[column], prefix= column, dtype=int)
  else:
      return df[[column]]

dummies = create_dummies(data, 'Housing_Tenure')
data= data.join(dummies)
data

,zipcode,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,hasCooling,hasGarage,hasHeating,hasSpa,...,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,Housing_Tenure,Housing_Tenure_Ownership,Housing_Tenure_Rental
17,78717,30.486408,-97.794724,2.21,0,True,True,False,True,False,...,3.266667,7.666667,1259,14,6.0,5,2,Rental,0,1
18,78717,30.494375,-97.796516,2.21,2,True,True,True,True,False,...,1.933333,8.333333,1481,16,4.0,5,2,Rental,0,1
19,78724,30.255707,-97.576958,1.98,2,False,True,True,True,False,...,2.433333,2.666667,1478,13,2.0,3,1,Ownership,1,0
20,78717,30.495638,-97.797874,2.21,0,True,True,False,True,False,...,1.900000,8.333333,1481,16,2.0,3,1,Rental,0,1
21,78717,30.490257,-97.791740,2.21,2,True,True,True,True,False,...,3.266667,7.666667,1259,14,4.0,4,2,Rental,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15166,78748,30.161997,-97.816345,1.98,6,False,True,True,True,False,...,0.900000,3.333333,1424,14,2.0,3,1,Rental,0,1
15167,78741,30.240223,-97.772835,1.98,2,True,True,True,True,True,...,1.900000,6.666667,1226,16,4.0,3,3,Rental,0,1
15168,78704,30.244835,-97.761124,1.98,2,False,True,True,True,False,...,0.800000,5.000000,889,13,3.0,3,2,Rental,0,1
15169,78722,30.281569,-97.717789,1.98,0,False,True,False,True,False,...,1.300000,6.666667,1057,15,2.0,4,1,Rental,0,1


In [19]:
data.select_dtypes(exclude='number').columns

Index(['hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'latest_saledate', 'Housing_Tenure'],
      dtype='object')

In [20]:
zp = pd.read_csv("/content/uszips.csv")
zp['zip']

,zip
0,601
1,602
2,603
3,606
4,610
...,...
33778,99923
33779,99925
33780,99926
33781,99927


In [21]:
zp[zp['city']== 'Austin']['zip'].nunique()

51

In [22]:
########### CHECKING IF MERGING IS RIGHT #########
missing_count = (data['zipcode'].isin(zp['zip']) == False).sum()
print(missing_count)

0


In [23]:
import pandas as pd
zp = zp.set_index('zip')
zp.head()

,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
zip,,,,,,,,,,,,,,,,,
601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16721.0,100.2,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37510.0,477.6,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,48317.0,543.1,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5435.0,47.3,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25413.0,264.4,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.82, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [24]:
data['population'] = data['zipcode'].map(zp['population'])
data['density'] = data['zipcode'].map(zp['density'])

In [25]:
data[data['population'].isna()]['population'].sum()

np.float64(0.0)

In [26]:
#USELESS FEATURES ## BUT WE DROP "city" and "zipcode" after using them. ## no dropping yet
useless_features_2= ['zipcode','zpid', 'city', 'streetAddress', 'description','homeImage','homeType', 'latestPriceSource']

In [27]:
df_1 = data.drop(columns=useless_features_2, errors= 'ignore')
df_1.isna().sum()

,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0
hasView,0


In [28]:
df_1['latest_saledate'] = df_1['latest_saledate'].astype(str).str.strip()
df_1['latest_saledate'] = pd.to_datetime(df_1['latest_saledate'], errors='coerce')
print(df_1['latest_saledate'].isna().sum())

0


In [29]:
df_1['latest_saledate'].head()

,latest_saledate
17,2018-08-28
18,2018-12-03
19,2020-12-22
20,2020-01-10
21,2020-12-07


In [30]:
df_1 = df_1.sort_values('latest_saledate').reset_index(drop=True)
df_1['latest_saledate']

,latest_saledate
0,2018-01-22
1,2018-01-22
2,2018-01-22
3,2018-01-23
4,2018-01-23
...,...
13000,2021-01-27
13001,2021-01-28
13002,2021-01-29
13003,2021-01-29


In [31]:
 # We have 958 days of analysis
 df_1['latest_saledate'].value_counts().sort_index().isna().sum()

np.int64(0)

In [32]:
df_1.select_dtypes(include= object)


,Housing_Tenure
0,Rental
1,Rental
2,Rental
3,Rental
4,Rental
...,...
13000,Rental
13001,Ownership
13002,Ownership
13003,Rental


In [33]:
# FINAL CHECK BEFORE MERGING
df_1.isna().sum()

,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0
hasView,0


In [34]:
# Get Mortgage rate
import pandas as pd
MTG = pd.read_csv('/content/MORTGAGE30US.csv')

print(MTG.shape)

MPPSQ = pd.read_csv('/content/MEDLISPRIPERSQUFEE12420.csv')
print(MPPSQ.dtypes)

(160, 2)
observation_date           object
MEDLISPRIPERSQUFEE12420     int64
dtype: object


In [35]:
############################################################################### Tranforming Mortgage Data #############################################################################################

In [36]:
################## TREATING DATES ON EXTERNAL MACRO VARIABLES ##################
def parse_obs_date(date_str):
    d = str(date_str).strip().split()[0]
    if "/" in d:
        return pd.to_datetime(d, format='%d/%m/%Y', errors='raise')
    else:
        return pd.to_datetime(d, errors='raise')

MTG['observation_date'] = MTG['observation_date'].apply(parse_obs_date)
MPPSQ['observation_date'] = MPPSQ['observation_date'].apply(parse_obs_date)
MTG['MORTGAGE30US'].dtypes

dtype('float64')

In [37]:
print(MTG.isna().sum())
print(MPPSQ.isna().sum())

observation_date    0
MORTGAGE30US        0
dtype: int64
observation_date           0
MEDLISPRIPERSQUFEE12420    0
dtype: int64


In [38]:
df_2 = df_1.drop(columns='Housing_Tenure', errors= 'ignore')
df_2

,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,hasCooling,hasGarage,hasHeating,hasSpa,hasView,...,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,Housing_Tenure_Ownership,Housing_Tenure_Rental,population,density
0,30.443314,-97.802513,1.98,0,True,True,False,True,False,False,...,8.666667,1632,16,3.0,4,1,0,1,29322.0,959.6
1,30.197430,-97.741470,1.98,0,False,True,False,True,False,False,...,3.000000,794,12,2.0,3,1,0,1,51069.0,872.3
2,30.157705,-97.782372,1.98,2,True,True,True,True,False,False,...,5.333333,1057,14,3.0,3,2,0,1,51069.0,872.3
3,30.398659,-97.696739,1.98,2,False,True,True,True,False,False,...,4.000000,1073,13,3.0,4,2,0,1,50592.0,2149.7
4,30.191980,-97.734581,1.98,1,False,True,True,True,False,False,...,3.666667,1287,14,2.0,3,1,0,1,51069.0,872.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13000,30.244835,-97.761124,1.98,2,False,True,True,True,False,False,...,5.000000,889,13,3.0,3,2,0,1,50479.0,2223.8
13001,30.367903,-97.653358,1.98,4,True,True,True,True,True,False,...,4.333333,1066,12,3.0,4,2,1,0,31410.0,801.1
13002,30.357496,-97.642845,1.98,0,True,True,False,True,False,False,...,4.333333,1066,12,2.0,4,1,1,0,31410.0,801.1
13003,30.424355,-97.726814,1.98,2,False,True,True,True,True,False,...,6.666667,1486,15,2.0,3,1,0,1,30894.0,1535.4


In [39]:
##################################.MERGE. ####################################

import pandas as pd

df_2['latest_saledate'] = pd.to_datetime(df_2['latest_saledate'], errors='coerce')
df_2 = df_2.sort_values('latest_saledate')

# Read and prepare the MTG rates
MTG['observation_date'] = pd.to_datetime(MTG['observation_date'], format='%d/%m/%Y', errors='coerce')
MTG = MTG.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MTG rate available
df_2 = pd.merge_asof(df_2, MTG, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories',
       'Housing_Tenure_Ownership', 'Housing_Tenure_Rental', 'population',
       'density', 'MORTGAGE30US'],
      dtype='object')

In [40]:
# read variable, sort variable on date
MPPSQ['observation_date'] = pd.to_datetime(MPPSQ['observation_date'], format='%d/%m/%Y', errors='coerce')
MPPSQ = MPPSQ.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MPPSQ available
df_2 = pd.merge_asof(df_2, MPPSQ, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if you no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories',
       'Housing_Tenure_Ownership', 'Housing_Tenure_Rental', 'population',
       'density', 'MORTGAGE30US', 'MEDLISPRIPERSQUFEE12420'],
      dtype='object')

In [41]:
df_2[['latest_saledate','MORTGAGE30US','MEDLISPRIPERSQUFEE12420']]

,latest_saledate,MORTGAGE30US,MEDLISPRIPERSQUFEE12420
0,2018-01-22,4.04,163
1,2018-01-22,4.04,163
2,2018-01-22,4.04,163
3,2018-01-23,4.04,163
4,2018-01-23,4.04,163
...,...,...,...
13000,2021-01-27,2.77,226
13001,2021-01-28,2.73,226
13002,2021-01-29,2.73,226
13003,2021-01-29,2.73,226


In [42]:
df_2.shape

(13005, 45)

In [43]:
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories',
       'Housing_Tenure_Ownership', 'Housing_Tenure_Rental', 'population',
       'density', 'MORTGAGE30US', 'MEDLISPRIPERSQUFEE12420'],
      dtype='object')

In [44]:
df_2['latestPrice_log'] = np.log(df_2['latestPrice'])
df_2.shape

(13005, 46)

In [45]:
df_2 = df_2.drop(columns= 'latestPrice', errors='ignore')
df_2.shape

(13005, 45)

In [46]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [47]:
# change path as desired, use the same path in the model file
df_2.to_pickle("/content/drive/MyDrive/lions_den_data/MODEL & DATA_CLEANING/df_2_trial2.pkl")

In [ ]:
## run -> on next script -> df_2 = pd.read_pickle("/content/drive/MyDrive/lions_den_data/MODEL & DATA_CLEANING/df_2_trial2.pkl")
